In [79]:
#Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from transformers import BertTokenizer
import numpy as np

In [80]:
# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Shape: (batch_size, 1, input_size)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Select the output of the last time step
        output = self.fc(lstm_out)
        return output


In [81]:
# Define the dataset
class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input': torch.tensor(self.encodings[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [82]:
# Define the dataset manually
train_texts = [
    "Dear Customer, Your account has been compromised. Please click on the following link to verify your identity.",
    "Hello, I am reaching out to inform you that you've won a lottery! Please provide your bank details to claim your prize.",
    "Dear User, Your account password has expired. Click the link below to reset it.",
    "Hi there, We've noticed unusual activity on your account. Please confirm your login details to secure your account.",
    "Dear Customer, Your subscription has been canceled. Click here to reinstate it.",
    "Dear Valued Customer, We have detected unauthorized transactions on your account. Please login to verify.",
    "Hello, This is a reminder to update your payment information. Failure to do so may result in service suspension.",
    "Dear User, Congratulations! You've been selected for a special offer. Click here to claim your reward.",
    "Hi, Your account is locked due to suspicious activity. Please provide additional information to unlock it.",
    "Dear Account Holder, Your account is suspended due to violation of our terms of service. Please contact support for assistance.",
    "Attention, Your account has been flagged for security reasons. Please verify your information to prevent further issues.",
    "Dear Customer, We're upgrading our systems. Kindly update your account information for uninterrupted service.",
    "Hello, Your recent purchase requires confirmation. Please review the details and validate the transaction.",
    "Dear User, You've reached your storage limit. Upgrade now to access additional space for your files.",
    "Hi there, Your trial period is ending soon. Subscribe now to continue enjoying premium features.",
    "Dear Valued Customer, Our records indicate an expired credit card on file. Please update your payment method.",
    "Attention, We've detected malware on your device. Click here for instructions to remove it.",
    "Dear Customer, Your recent login was from an unrecognized device. Review and secure your account.",
    "Hello, Your account recovery options need updating. Take action now to ensure seamless access.",
    "Dear User, You've been selected to participate in a survey. Your feedback is highly appreciated.",
    "Hi, We're conducting maintenance tonight. Expect temporary service disruption during the hours of 12 AM to 4 AM.",
    "Dear Account Holder, We've received reports of phishing attempts. Be cautious and report any suspicious emails.",
    "Attention, Your account is pending verification. Complete the process to unlock full access.",
    "Dear Customer, Your subscription plan is due for renewal. Update your payment details to continue enjoying our service.",
    "Hello, You've earned reward points! Log in to view your balance and redeem exciting offers.",
    "Dear Valued Customer, Your account has been credited with a bonus. Thank you for your continued support.",
    "Hi there, We've upgraded our security features. Ensure your account is protected by updating your password.",
    "Dear User, Your account activity seems unusual. Secure your account by changing your password immediately.",
    "Attention, Your account is inactive. Log in within the next 7 days to prevent it from being deactivated.",
    "Dear Account Holder, Our terms of service have been updated. Review them to continue using our platform.",
    "Hello, Your subscription has expired. Renew now to avoid interruption of your service.",
    "Dear Customer, Your recent transaction requires confirmation. Please log in to authorize the payment.",
    "Hi, We're sorry to see you go! Let us know how we can improve our service in the future.",
    "Dear Valued Customer, We've launched new features! Explore them now for an enhanced experience.",
    "Attention, Your account settings need attention. Review and update your preferences as needed.",
    "Dear User, We've noticed a decrease in your activity. Is there anything we can assist you with?",
    "Hello, Your account balance is low. Add funds now to continue using our services.",
    "Dear Account Holder, Your account has been upgraded to premium status. Enjoy exclusive benefits!",
    "Hi there, We're conducting a security audit. Your cooperation may be required to ensure account safety.",
    "Dear Customer, We've received your inquiry. A representative will get back to you shortly.",
    "Attention, We've detected outdated software on your device. Update now to avoid security vulnerabilities.",
    "Dear User, Your privacy settings have been updated. Review and adjust them according to your preferences.",
    "Hello, Your account recovery options have been updated. Ensure they are up-to-date for smooth access.",
    "Dear Valued Customer, You're one step away from completing your registration. Verify your email to proceed.",
    "Hi, Your account information has been updated successfully. If you did not make these changes, please contact us immediately.",
    "Attention, Your account is temporarily suspended. Contact support to resolve the issue.",
    "Dear Account Holder, Thank you for your recent purchase. For any inquiries, feel free to reach out to us.",
    "Hello, Your subscription has been renewed successfully. Enjoy uninterrupted access to our services.",
    "Dear Customer, We've added new security measures. Please review and acknowledge them for continued protection.",
    "Hi there, We've received your complaint. Our team is investigating the matter and will provide an update soon.",
    "Attention, Your account is under review. Expect an email from us with further instructions.",
    "Dear User, Your account settings have been reset to default. Please adjust them according to your preferences.",
    "Hello, Your feedback is important to us! Take a moment to complete our satisfaction survey.",
    "Dear Valued Customer, Our server will undergo maintenance tomorrow. Expect brief service disruptions.",
    "Hi, We've detected suspicious activity on your account. Take immediate action to secure it.",
    "Attention, Your account password has been changed. If you did not authorize this change, please contact support immediately.",
    "Dear Account Holder, Thank you for your recent payment. Your account is now up-to-date.",
    "Hello, Your subscription has been upgraded to a premium plan. Enjoy enhanced features and benefits.",
    "Dear Customer, Your request for assistance has been received. Our team will address it shortly.",
    "Hi there, We've noticed you haven't logged in for a while. Log in now to keep your account active.",
    "Attention, Your account has been inactive for an extended period. Log in to prevent it from being deactivated.",
    "Dear User, Your account has been credited with a bonus reward. Enjoy!",
    "Hello, Your account access has been restored. We apologize for any inconvenience caused.",
    "Dear Valued Customer, We're excited to announce upcoming features! Stay tuned for more information.",
    "Hi, Your account has been temporarily locked. Follow the instructions to regain access.",
    "Attention, Your account subscription is expiring soon. Renew now to avoid service interruption.",
    "Dear Account Holder, We've detected multiple failed login attempts on your account. Secure it by changing your password.",
    "Hello, Your recent activity indicates a possible security breach. Review and secure your account immediately.",
    "Dear Customer, We appreciate your feedback! We're constantly working to improve our services.",
    "Hi there, Your account details have been updated successfully. If you did not authorize these changes, please contact us.",
    "Attention, We've received reports of phishing emails targeting our customers. Be vigilant and report any suspicious activity.",
    "Dear User, Your account status has been updated. Log in to view the changes.",
    "Hello, We've updated our privacy policy. Review it to understand how we protect your data.",
    "Dear Valued Customer, We've received your inquiry and will respond within 24 hours.",
    "Hi, Your account has been flagged for suspicious activity. Verify your identity to unlock it.",
    "Attention, Your account balance is low. Add funds to continue using our services.",
    "Dear Account Holder, We've detected a login from a new device. If this was you, no action is needed.",
    "Hello, Your account recovery options have been updated successfully.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy continued access to our services.",
    "Hi there, We're experiencing technical difficulties. We apologize for any inconvenience caused.",
    "Attention, Your account is at risk. Take immediate action to secure it.",
    "Dear User, Thank you for your recent purchase. For any inquiries, please contact our support team.",
    "Hello, Your account has been credited with a referral bonus. Refer more friends to earn additional rewards!",
    "Dear Valued Customer, We've received your payment. Your account is now active.",
    "Hi, We're conducting a customer satisfaction survey. Your feedback helps us improve our services.",
    "Attention, Your account is locked due to suspicious activity. Contact support to regain access.",
    "Dear Account Holder, We're sorry to see you go! Let us know how we can improve our services.",
    "Hello, Your account has been upgraded to a premium membership. Enjoy exclusive benefits!",
    "Dear Customer, We've detected unusual activity on your account. Secure it by changing your password.",
    "Hi there, Your subscription plan is due for renewal. Update your payment details to avoid service interruption.",
    "Attention, Your account has been compromised. Take immediate action to secure it.",
    "Dear User, We've received your request for assistance. A support representative will be in touch shortly.",
    "Hello, Your recent transaction requires confirmation. Log in to authorize the payment.",
    "Dear Valued Customer, We appreciate your continued support. Thank you!",
    "Hi, We're conducting maintenance tonight. Expect temporary service disruptions.",
    "Attention, We've detected malware on your device. Click here for instructions to remove it.",
    "Dear Account Holder, Your account is temporarily suspended. Contact support for assistance.",
    "Hello, Your account recovery options need updating. Take action now to ensure seamless access.",
    "Dear Customer, Your account has been credited with bonus points. Enjoy!",
    "Hi there, We've upgraded our security features. Ensure your account is protected by updating your password.",
    "Attention, Your account settings have been updated. Review and adjust them as needed.",
    "Dear User, We've noticed a decrease in your activity. Is there anything we can assist you with?",
    "Hello, Your account balance is low. Add funds now to continue using our services.",
    "Dear Account Holder, Your account has been upgraded to premium status. Enjoy exclusive benefits!",
    "Hi, We're conducting a security audit. Your cooperation may be required to ensure account safety.",
    "Attention, Your account is inactive. Log in within the next 7 days to prevent it from being deactivated.",
    "Dear Customer, Our terms of service have been updated. Review them to continue using our platform.",
    "Hello, Your subscription has expired. Renew now to avoid interruption of your service.",
    "Dear Valued Customer, Your recent transaction requires confirmation. Please log in to authorize the payment.",
    "Hi, We're sorry to see you go! Let us know how we can improve our service in the future.",
    "Attention, Your account has been pending verification. Complete the process to unlock full access.",
    "Dear Account Holder, We've received reports of phishing attempts. Be cautious and report any suspicious emails.",
    "Hello, You've earned reward points! Log in to view your balance and redeem exciting offers.",
    "Dear Valued Customer, We've launched new features! Explore them now for an enhanced experience.",
    "Hi there, We've received your complaint. Our team is investigating the matter and will provide an update soon.",
    "Attention, Your account is under review. Expect an email from us with further instructions.",
    "Dear User, Your account settings have been reset to default. Please adjust them according to your preferences.",
    "Hello, Your feedback is important to us! Take a moment to complete our satisfaction survey.",
    "Dear Valued Customer, Our server will undergo maintenance tomorrow. Expect brief service disruptions.",
    "Hi, We've detected suspicious activity on your account. Take immediate action to secure it.",
    "Attention, Your account password has been changed. If you did not authorize this change, please contact support immediately.",
    "Dear Account Holder, Thank you for your recent payment. Your account is now up-to-date.",
    "Hello, Your subscription has been upgraded to a premium plan. Enjoy enhanced features and benefits.",
    "Dear Customer, Your request for assistance has been received. Our team will address it shortly.",
    "Hi there, We've noticed you haven't logged in for a while. Log in now to keep your account active.",
    "Attention, Your account has been inactive for an extended period. Log in to prevent it from being deactivated.",
    "Dear User, Your account has been credited with a bonus reward. Enjoy!",
    "Hello, Your account access has been restored. We apologize for any inconvenience caused.",
    "Dear Valued Customer, We're excited to announce upcoming features! Stay tuned for more information.",
    "Hi, Your account has been temporarily locked. Follow the instructions to regain access.",
    "Attention, Your account subscription is expiring soon. Renew now to avoid service interruption.",
    "Dear Account Holder, We've detected multiple failed login attempts on your account. Secure it by changing your password.",
    "Hello, Your recent activity indicates a possible security breach. Review and secure your account immediately.",
    "Dear Customer, We appreciate your feedback! We're constantly working to improve our services.",
    "Hi there, Your account details have been updated successfully. If you did not authorize these changes, please contact us.",
    "Attention, We've received reports of phishing emails targeting our customers. Be vigilant and report any suspicious activity.",
    "Dear User, Your account status has been updated. Log in to view the changes.",
    "Hello, We've updated our privacy policy. Review it to understand how we protect your data.",
    "Dear Valued Customer, We've received your inquiry and will respond within 24 hours.",
    "Hi, Your account has been flagged for suspicious activity. Verify your identity to unlock it.",
    "Attention, Your account balance is low. Add funds to continue using our services.",
    "Dear Account Holder, We've detected a login from a new device. If this was you, no action is needed.",
    "Hello, Your account recovery options have been updated successfully.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy continued access to our services.",
    "Hi there, We're experiencing technical difficulties. We apologize for any inconvenience caused.",
    "Attention, Your account is at risk. Take immediate action to secure it.",
    "Dear User, Thank you for your recent purchase. For any inquiries, please contact our support team.",
    "Hello, Your account has been credited with a referral bonus. Refer more friends",
    "Dear Customer, We hope you're enjoying our services! Please take a moment to provide your feedback.",
    "Hello, This is a reminder to renew your subscription before it expires. Click here to proceed.",
    "Dear User, Thank you for your recent purchase! Please review your order details.",
    "Hi there, Our team is here to assist you! Contact us if you have any questions or concerns.",
    "Dear Valued Customer, Enjoy free shipping on all orders placed today! Limited time offer.",
    "Hello, We've added new features to our platform! Check them out now.",
    "Dear User, Your account has been upgraded to premium! Enjoy exclusive benefits.",
    "Hi, Don't miss out on our latest promotions! Subscribe to our newsletter for updates.",
    "Dear Account Holder, Thank you for choosing our services. We appreciate your business.",
    "Hello, Your account balance is low. Please reload funds to continue using our services.",
    "Dear Customer, Thank you for your loyalty! Here's a special discount code just for you.",
    "Hi there, Have you seen our new collection? Check it out now for the latest trends.",
    "Dear User, Your feedback matters to us! Please take a moment to complete our survey.",
    "Hello, Your account has been successfully activated. Welcome to our platform!",
    "Dear Valued Customer, Get ready for our upcoming sale event! Stay tuned for exciting deals.",
    "Hi, We've noticed you haven't visited us in a while. Here's a special offer to welcome you back.",
    "Dear Account Holder, Your payment has been received. Thank you for your prompt response.",
    "Hello, Your recent order is on its way! Track your package with the provided link.",
    "Dear Customer, It's our anniversary! Join us in celebrating with exclusive discounts.",
    "Hi there, Need assistance with your account? Our support team is available 24/7 to help.",
    "Dear User, Don't forget to update your profile with your latest information for personalized offers.",
    "Hello, Your account has been credited with reward points! Redeem them for great rewards.",
    "Dear Valued Customer, We're thrilled to have you onboard! Here's a token of our appreciation.",
    "Hi, We're excited to announce our new partnership. Stay tuned for exciting collaborations.",
    "Dear Account Holder, Your account status has been upgraded. Enjoy enhanced features!",
    "Hello, Your recent inquiry has been resolved. Let us know if you need any further assistance.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy uninterrupted access!",
    "Hi there, Have you seen our latest blog post? Click here to read about our latest updates.",
    "Dear User, We've extended our sale! Don't miss out on the chance to save big.",
    "Hello, Your account has been verified successfully. Thank you for confirming your details.",
    "Dear Valued Customer, Your feedback helps us improve! Share your thoughts with us today.",
    "Hi, We've updated our terms of service. Please review them to continue using our platform.",
    "Dear Account Holder, Congratulations! You've unlocked a special offer. Claim it now.",
    "Hello, Your recent transaction has been processed successfully. Thank you for your business.",
    "Dear Customer, Need help getting started? Check out our beginner's guide for tips and tricks.",
    "Hi there, Our support team is here to help. Reach out to us with any questions or concerns.",
    "Dear User, Your account has been credited with a bonus. Enjoy the extra perks!",
    "Hello, Your order is confirmed! Sit back and relax while we prepare it for shipping.",
    "Dear Valued Customer, We're excited to announce our upcoming event. Save the date!",
    "Hi, Don't forget to follow us on social media for the latest updates and exclusive content.",
    "Dear Account Holder, Your account settings have been updated. Review them for accuracy.",
    "Hello, Your recent feedback has been received. Thank you for helping us serve you better.",
    "Dear Customer, Your subscription plan has been upgraded. Enjoy premium benefits!",
    "Hi there, We've launched a new feature! Check it out and let us know what you think.",
    "Dear User, Your account has been credited with cashback rewards. Start redeeming today!",
    "Hello, Your recent purchase qualifies you for a special offer. Don't miss out!",
    "Dear Valued Customer, Here's a sneak peek of our upcoming products. Stay tuned for more!",
    "Hi, We're here to help you make the most of your membership. Contact us for tips and advice.",
    "Dear Account Holder, Thank you for your continued support. We value your partnership.",
    "Hello, Your account status has been updated. Log in to view the changes.",
    "Dear Customer, Our latest collection is now available! Explore it online or in-store.",
    "Hi there, We've extended our customer support hours. Get in touch with us at any time.",
    "Dear User, We've received your inquiry and will respond shortly. Thank you for reaching out.",
    "Hello, Your recent order is being processed. We'll notify you once it's shipped.",
    "Dear Valued Customer, We're celebrating our anniversary with exclusive discounts just for you.",
    "Hi, We're excited to announce our partnership with a renowned brand. Stay tuned for updates!",
    "Dear Account Holder, Your payment has been received. Thank you for your prompt response.",
    "Hello, Your recent order is on its way! Track your package with the provided link.",
    "Dear Customer, It's our anniversary! Join us in celebrating with exclusive discounts.",
    "Hi there, Need assistance with your account? Our support team is available 24/7 to help.",
    "Dear User, Don't forget to update your profile with your latest information for personalized offers.",
    "Hello, Your account has been credited with reward points! Redeem them for great rewards.",
    "Dear Valued Customer, We're thrilled to have you onboard! Here's a token of our appreciation.",
    "Hi, We're excited to announce our new partnership. Stay tuned for exciting collaborations.",
    "Dear Account Holder, Your account status has been upgraded. Enjoy enhanced features!",
    "Hello, Your recent inquiry has been resolved. Let us know if you need any further assistance.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy uninterrupted access!",
    "Hi there, Have you seen our latest blog post? Click here to read about our latest updates.",
    "Dear User, We've extended our sale! Don't miss out on the chance to save big.",
    "Hello, Your account has been verified successfully. Thank you for confirming your details.",
    "Dear Valued Customer, Your feedback helps us improve! Share your thoughts with us today.",
    "Hi, We've updated our terms of service. Please review them to continue using our platform.",
    "Dear Account Holder, Congratulations! You've unlocked a special offer. Claim it now.",
    "Hello, Your recent transaction has been processed successfully. Thank you for your business.",
    "Dear Customer, Need help getting started? Check out our beginner's guide for tips and tricks.",
    "Hi there, Our support team is here to help. Reach out to us with any questions or concerns.",
    "Dear User, Your account has been credited with a bonus. Enjoy the extra perks!",
    "Hello, Your order is confirmed! Sit back and relax while we prepare it for shipping.",
    "Dear Valued Customer, We're excited to announce our upcoming event. Save the date!",
    "Hi, Don't forget to follow us on social media for the latest updates and exclusive content.",
    "Dear Account Holder, Your account settings have been updated. Review them for accuracy.",
    "Hello, Your recent feedback has been received. Thank you for helping us serve you better.",
    "Dear Customer, Your subscription plan has been upgraded. Enjoy premium benefits!",
    "Hi there, We've launched a new feature! Check it out and let us know what you think.",
    "Dear User, Your account has been credited with cashback rewards. Start redeeming today!",
    "Hello, Your recent purchase qualifies you for a special offer. Don't miss out!",
    "Dear Valued Customer, Here's a sneak peek of our upcoming products. Stay tuned for more!",
    "Hi, We're here to help you make the most of your membership. Contact us for tips and advice.",
    "Dear Account Holder, Thank you for your continued support. We value your partnership.",
    "Hello, Your account status has been updated. Log in to view the changes.",
    "Dear Customer, Our latest collection is now available! Explore it online or in-store.",
    "Hi there, We've extended our customer support hours. Get in touch with us at any time.",
    "Dear User, We've received your inquiry and will respond shortly. Thank you for reaching out.",
    "Hello, Your recent order is being processed. We'll notify you once it's shipped.",
    "Dear Valued Customer, We're celebrating our anniversary with exclusive discounts just for you.",
    "Hi, We're excited to announce our partnership with a renowned brand. Stay tuned for updates!",
    "Dear Account Holder, Your payment has been received. Thank you for your prompt response.",
    "Hello, Your recent order is on its way! Track your package with the provided link.",
    "Dear Customer, It's our anniversary! Join us in celebrating with exclusive discounts.",
    "Hi there, Need assistance with your account? Our support team is available 24/7 to help.",
    "Dear User, Don't forget to update your profile with your latest information for personalized offers.",
    "Hello, Your account has been credited with reward points! Redeem them for great rewards.",
    "Dear Valued Customer, We're thrilled to have you onboard! Here's a token of our appreciation.",
    "Hi, We're excited to announce our new partnership. Stay tuned for exciting collaborations.",
    "Dear Account Holder, Your account status has been upgraded. Enjoy enhanced features!",
    "Hello, Your recent inquiry has been resolved. Let us know if you need any further assistance.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy uninterrupted access!",
    "Hi there, Have you seen our latest blog post? Click here to read about our latest updates.",
    "Dear User, We've extended our sale! Don't miss out on the chance to save big.",
    "Hello, Your account has been verified successfully. Thank you for confirming your details.",
    "Dear Valued Customer, Your feedback helps us improve! Share your thoughts with us today.",
    "Hi, We've updated our terms of service. Please review them to continue using our platform.",
    "Dear Account Holder, Congratulations! You've unlocked a special offer. Claim it now.",
    "Hello, Your recent transaction has been processed successfully. Thank you for your business.",
    "Dear Customer, Need help getting started? Check out our beginner's guide for tips and tricks.",
    "Hi there, Our support team is here to help. Reach out to us with any questions or concerns.",
    "Dear User, Your account has been credited with a bonus. Enjoy the extra perks!",
    "Hello, Your order is confirmed! Sit back and relax while we prepare it for shipping.",
    "Dear Valued Customer, We're excited to announce our upcoming event. Save the date!",
    "Hi, Don't forget to follow us on social media for the latest updates and exclusive content.",
    "Dear Account Holder, Your account settings have been updated. Review them for accuracy.",
    "Hello, Your recent feedback has been received. Thank you for helping us serve you better.",
    "Dear Customer, Your subscription plan has been upgraded. Enjoy premium benefits!",
    "Hi there, We've launched a new feature! Check it out and let us know what you think.",
    "Dear User, Your account has been credited with cashback rewards. Start redeeming today!",
    "Hello, Your recent purchase qualifies you for a special offer. Don't miss out!",
    "Dear Valued Customer, Here's a sneak peek of our upcoming products. Stay tuned for more!",
    "Hi, We're here to help you make the most of your membership. Contact us for tips and advice.",
    "Dear Account Holder, Thank you for your continued support. We value your partnership.",
    "Hello, Your account status has been updated. Log in to view the changes.",
    "Dear Customer, Our latest collection is now available! Explore it online or in-store.",
    "Hi there, We've extended our customer support hours. Get in touch with us at any time.",
    "Dear User, We've received your inquiry and will respond shortly. Thank you for reaching out.",
    "Hello, Your recent order is being processed. We'll notify you once it's shipped.",
    "Dear Valued Customer, We're celebrating our anniversary with exclusive discounts just for you.",
    "Hi, We're excited to announce our partnership with a renowned brand. Stay tuned for updates!",
    "Dear Account Holder, Your payment has been received. Thank you for your prompt response.",
    "Hello, Your recent order is on its way! Track your package with the provided link.",
    "Dear Customer, It's our anniversary! Join us in celebrating with exclusive discounts.",
    "Hi there, Need assistance with your account? Our support team is available 24/7 to help.",
    "Dear User, Don't forget to update your profile with your latest information for personalized offers.",
    "Hello, Your account has been credited with reward points! Redeem them for great rewards.",
    "Dear Valued Customer, We're thrilled to have you onboard! Here's a token of our appreciation.",
    "Hi, We're excited to announce our new partnership. Stay tuned for exciting collaborations.",
    "Dear Account Holder, Your account status has been upgraded. Enjoy enhanced features!",
    "Hello, Your recent inquiry has been resolved. Let us know if you need any further assistance.",
    "Dear Customer, Your subscription has been renewed automatically. Enjoy uninterrupted access!",
    "Hi there, Have you seen our latest blog post? Click here to read about our latest updates."
]

In [107]:
# Define labels for train_texts (0 for fake)
train_labels = [0] * len(train_texts)

In [ ]:
# Manually split the dataset into train and validation sets
from sklearn.model_selection import train_test_split
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)


# print(validation_texts)
# print(train_texts)

In [109]:
# Tokenize texts
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)['input_ids']
validation_encodings = tokenizer(validation_texts, truncation=True, padding=True)['input_ids']

In [145]:
# Define the number of folds for cross-validation
num_folds = 3
kf = KFold(n_splits=num_folds)

In [146]:
# Define lists to store evaluation metrics
val_accuracies = []
val_losses = []

In [160]:
# Iterate over the folds
for fold, (train_index, val_index) in enumerate(kf.split(train_encodings)):
    print(f'Fold [{fold+1}/{num_folds}]')

    # Split data into train and validation sets for this fold
    train_inputs = [train_encodings[i] for i in train_index]
    train_labels_fold = [train_labels[i] for i in train_index]

    val_inputs = [train_encodings[i] for i in val_index]
    val_labels_fold = [train_labels[i] for i in val_index]

    # Define data loaders for training and validation sets
    train_dataset_fold = EmailDataset(train_inputs, train_labels_fold)
    train_loader_fold = DataLoader(train_dataset_fold, batch_size=32, shuffle=True)

    val_dataset_fold = EmailDataset(val_inputs, val_labels_fold)
    val_loader_fold = DataLoader(val_dataset_fold, batch_size=16, shuffle=False)

    # Initialize the model, loss function, and optimizer
    input_size = len(train_encodings[0])
    hidden_size = 80
    output_size = 2  # Real vs Fake
    model = LSTMClassifier(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00024)

    # Training the model
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch in train_loader_fold:
            inputs = batch['input']
            labels = batch['label']

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in val_loader_fold:
                inputs = batch['input']
                labels = batch['label']
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate validation accuracy and loss for this fold
        val_accuracy_fold = (correct / total) * 100
        val_loss_fold = val_loss / len(val_loader_fold)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Validation Loss: {val_loss_fold:.4f}, '
              f'Validation Accuracy: {val_accuracy_fold:.4f}%')

    # Store validation accuracy and loss for this fold
    val_accuracies.append(val_accuracy_fold)
    val_losses.append(val_loss_fold)

Fold [1/3]
Epoch [1/20], Validation Loss: 0.6786, Validation Accuracy: 58.7500%
Epoch [2/20], Validation Loss: 0.6571, Validation Accuracy: 67.5000%
Epoch [3/20], Validation Loss: 0.6416, Validation Accuracy: 68.7500%
Epoch [4/20], Validation Loss: 0.6330, Validation Accuracy: 71.2500%
Epoch [5/20], Validation Loss: 0.6223, Validation Accuracy: 71.2500%
Epoch [6/20], Validation Loss: 0.6095, Validation Accuracy: 72.5000%
Epoch [7/20], Validation Loss: 0.6002, Validation Accuracy: 75.0000%
Epoch [8/20], Validation Loss: 0.5908, Validation Accuracy: 78.7500%
Epoch [9/20], Validation Loss: 0.5826, Validation Accuracy: 81.2500%
Epoch [10/20], Validation Loss: 0.5746, Validation Accuracy: 81.2500%
Epoch [11/20], Validation Loss: 0.5662, Validation Accuracy: 83.7500%
Epoch [12/20], Validation Loss: 0.5582, Validation Accuracy: 83.7500%
Epoch [13/20], Validation Loss: 0.5510, Validation Accuracy: 83.7500%
Epoch [14/20], Validation Loss: 0.5427, Validation Accuracy: 83.7500%
Epoch [15/20], Val

In [161]:
# Calculate mean and standard deviation of validation metrics
mean_val_accuracy = np.mean(val_accuracies)
mean_val_loss = np.mean(val_losses)
std_val_accuracy = np.std(val_accuracies)
std_val_loss = np.std(val_losses)

print(f'Mean Validation Accuracy: {mean_val_accuracy:.4f}% ± {std_val_accuracy:.4f}')
print(f'Mean Validation Loss: {mean_val_loss:.4f} ± {std_val_loss:.4f}')

Mean Validation Accuracy: 95.8155% ± 6.1920
Mean Validation Loss: 0.3919 ± 0.1268
